In [ ]:
'''
Plots are from PhD candidate Yinfei Yang at Imperial College London, with some modifications
'''
#settings of plots from Methods for Data Science guide for plotting at Imperial College London
import os
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
# we define a mix of fontsizes, for different parts of a plot
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

# example of how you can use these fontsizes to set a global configuration for matplotlib;
# you should assign them based on the specific appearance of the figures you are producing
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#setting seeds
seed = 6
random.seed(seed )
torch.manual_seed(seed )
np.random.seed(seed ) 

In [23]:
import pandas as pd
import plotly.express as px
import os
import re

# First plot: losses from multiple CSVs
losses_folder = 'losses'
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_multiFold.csv'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if filename.endswith(".csv"):
        match = re.search(filename_pattern, filename)
        if not match:
            continue
        n_heads = int(match.group(1))
        n_layers = int(match.group(2))
        d_k = int(match.group(3))
        dff = int(match.group(4))
        dropout = float(match.group(5))

        df_file = pd.read_csv(os.path.join(losses_folder, filename))
        df_file['n_heads'] = n_heads
        df_file['n_layers'] = n_layers
        df_file['d_k'] = d_k
        df_file['dff'] = dff
        df_file['dropout'] = dropout
        all_losses_data.append(df_file)

df_combined = pd.concat(all_losses_data, ignore_index=True)

# Select the best validation loss for each hyperparameter combination
df_plot = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout'])['val_loss'].idxmin()
].reset_index(drop=True)
df_plot = df_plot.rename(columns={'val_loss': 'Eval Loss'})

# Rename columns for display
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dff": "Feed Forward Dimension",
    "dropout": "Dropout"
})

custom_colors = [
    [0.0, "#083D77"],
    [0.5, "#9BC995"],
    [0.75, "#FC7753"],
    [1.0, "#9A031E"]
]

fig1 = px.parallel_coordinates(
    df_plot,
    color="Eval Loss",
    dimensions=[
        "Number of Layers",
        "Number of Heads",
        "d_k",
        "Feed Forward Dimension",
        "Dropout",
        "Eval Loss"
    ],
    color_continuous_scale=custom_colors,
    range_color=[df_plot["Eval Loss"].min(), df_plot["Eval Loss"].max()]
)

# Force same layout/font
fig1.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black")
)

fig1.show()

In [24]:
import pandas as pd
import plotly.express as px
import os
import re

losses_folder = 'losses'
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_multiFold.csv'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if filename.endswith(".csv"):
        match = re.search(filename_pattern, filename)
        if not match:
            continue
        n_heads = int(match.group(1))
        n_layers = int(match.group(2))
        d_k = int(match.group(3))
        dff = int(match.group(4))
        dropout = float(match.group(5))

        df_file = pd.read_csv(os.path.join(losses_folder, filename))
        df_file['n_heads'] = n_heads
        df_file['n_layers'] = n_layers
        df_file['d_k'] = d_k
        df_file['dff'] = dff
        df_file['dropout'] = dropout
        all_losses_data.append(df_file)

df_combined = pd.concat(all_losses_data, ignore_index=True)

# Best validation loss per hyperparameter combination
df_plot = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout'])['val_loss'].idxmin()
].reset_index(drop=True)
df_plot = df_plot.rename(columns={'val_loss': 'Best Eval Loss'})

# Rename columns
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dff": "Feed Forward Layer",
    "dropout": "Dropout"
})

# Highlight only the best overall combination
best_idx = df_plot["Best Eval Loss"].idxmin()
df_plot["BestFlag"] = 0
df_plot.loc[best_idx, "BestFlag"] = 1

fig2 = px.parallel_coordinates(
    df_plot,
    color="BestFlag",
    dimensions=[
        "Number of Heads",
        "Number of Layers",
        "d_k",
        "Feed Forward Layer",
        "Dropout",
        "Best Eval Loss"
    ],
    color_continuous_scale=[[0, "lightgray"], [1, "#9A031E"]],
    range_color=[0, 1]
)

# Hide colorbar and enforce consistent layout/font
fig2.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black"),coloraxis_showscale=False
)

fig2.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os
import re

# Folder with loss CSVs
losses_folder = 'losses2'

# Filename pattern (with fold)
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_fold(\d+)_multiFold.csv'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if not filename.endswith(".csv"):
        continue
    match = re.search(filename_pattern, filename)
    if not match:
        continue

    n_heads = int(match.group(1))
    n_layers = int(match.group(2))
    d_k = int(match.group(3))
    dff = int(match.group(4))
    dropout = float(match.group(5))
    fold = int(match.group(6))

    df_file = pd.read_csv(os.path.join(losses_folder, filename))
    df_file['n_heads'] = n_heads
    df_file['n_layers'] = n_layers
    df_file['d_k'] = d_k
    df_file['dff'] = dff
    df_file['dropout'] = dropout
    df_file['fold'] = fold
    all_losses_data.append(df_file)

# Combine all loss files
df_combined = pd.concat(all_losses_data, ignore_index=True)

# Best val_loss per fold
df_best_per_fold = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout', 'fold'])['val_loss'].idxmin()
].reset_index(drop=True)

# Average best loss across folds
df_plot = df_best_per_fold.groupby(
    ['n_heads', 'n_layers', 'd_k', 'dff', 'dropout']
)['val_loss'].mean().reset_index()

# Rename final column
df_plot = df_plot.rename(columns={'val_loss': 'Average Eval Loss'})

# Rename hyperparameter columns
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dropout": "Dropout"
})

# Sort by best loss
df_plot = df_plot.sort_values("Average Eval Loss", ascending=True).reset_index(drop=True)

# Parallel coordinates plot WITHOUT Feed Forward Dimension
custom_colors = [
    [0.0, "#083D77"],
    [0.5, "#9BC995"],
    [0.75, "#FC7753"],
    [1.0, "#9A031E"]
]

fig = px.parallel_coordinates(
    df_plot,
    color="Average Eval Loss",
    dimensions=[
        "Number of Layers",
        "Number of Heads",
        "d_k",
        "Dropout",
        "Average Eval Loss"
    ],
    color_continuous_scale=custom_colors,
    range_color=[df_plot["Average Eval Loss"].min(), df_plot["Average Eval Loss"].max()]
)

fig.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black")
)

fig.show()


In [26]:
import pandas as pd
import plotly.express as px
import os
import re

# Folder with loss CSVs
losses_folder = 'losses2'

# Filename pattern (with fold)
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_fold(\d+)_multiFold.csv'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if not filename.endswith(".csv"):
        continue
    match = re.search(filename_pattern, filename)
    if not match:
        continue

    n_heads = int(match.group(1))
    n_layers = int(match.group(2))
    d_k = int(match.group(3))
    dff = int(match.group(4))
    dropout = float(match.group(5))
    fold = int(match.group(6))

    df_file = pd.read_csv(os.path.join(losses_folder, filename))
    df_file['n_heads'] = n_heads
    df_file['n_layers'] = n_layers
    df_file['d_k'] = d_k
    df_file['dff'] = dff
    df_file['dropout'] = dropout
    df_file['fold'] = fold
    all_losses_data.append(df_file)

# Combine all loss files
df_combined = pd.concat(all_losses_data, ignore_index=True)

# Step 1: Best val_loss per fold
df_best_per_fold = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout', 'fold'])['val_loss'].idxmin()
].reset_index(drop=True)

# Step 2: Average best loss across folds
df_plot = df_best_per_fold.groupby(
    ['n_heads', 'n_layers', 'd_k', 'dff', 'dropout']
)['val_loss'].mean().reset_index()

# Rename final column
df_plot = df_plot.rename(columns={'val_loss': 'Average Eval Loss'})

# Rename hyperparameter columns
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dropout": "Dropout"
})

# Flag best configuration
best_idx = df_plot["Average Eval Loss"].idxmin()
df_plot["BestFlag"] = 0
df_plot.loc[best_idx, "BestFlag"] = 1

# Parallel coordinates plot WITHOUT Feed Forward Dimension
fig = px.parallel_coordinates(
    df_plot,
    color="BestFlag",
    dimensions=[
        "Number of Layers",
        "Number of Heads",
        "d_k",
        "Dropout",
        "Average Eval Loss"
    ],
    color_continuous_scale=[[0, "lightgray"], [1, "#9A031E"]],
    range_color=[0, 1]
)

fig.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black"),coloraxis_showscale=False
)

fig.show()


In [27]:
import os
import re
import pandas as pd
import plotly.express as px

# Folder with loss CSVs
losses_folder = 'losses3'

# New filename pattern (no fold inside filename anymore)
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_fiveFold\.csv'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if not filename.endswith(".csv"):
        continue
    match = re.search(filename_pattern, filename)
    if not match:
        continue

    n_heads = int(match.group(1))
    n_layers = int(match.group(2))
    d_k = int(match.group(3))
    dff = int(match.group(4))
    dropout = float(match.group(5))

    df_file = pd.read_csv(os.path.join(losses_folder, filename))
    df_file['n_heads'] = n_heads
    df_file['n_layers'] = n_layers
    df_file['d_k'] = d_k
    df_file['dff'] = dff
    df_file['dropout'] = dropout
    all_losses_data.append(df_file)

# Combine all loss files
df_combined = pd.concat(all_losses_data, ignore_index=True)

# Step 1: Best val_loss per fold
df_best_per_fold = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout', 'fold'])['val_loss'].idxmin()
].reset_index(drop=True)

# Step 2: Average best loss across folds
df_plot = df_best_per_fold.groupby(
    ['n_heads', 'n_layers', 'd_k', 'dff', 'dropout']
)['val_loss'].mean().reset_index()

# Rename final column
df_plot = df_plot.rename(columns={'val_loss': 'Average Eval Loss'})

# Rename hyperparameter columns
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dff": "Feed Forward Dimension",
    "dropout": "Dropout"
})

# Sort by best loss
df_plot = df_plot.sort_values("Average Eval Loss", ascending=True).reset_index(drop=True)

# Parallel coordinates plot
custom_colors = [
    [0.0, "#083D77"],
    [0.5, "#9BC995"],
    [0.75, "#FC7753"],
    [1.0, "#9A031E"]
]

fig = px.parallel_coordinates(
    df_plot,
    color="Average Eval Loss",
    dimensions=[
        "Number of Heads",
         "Number of Layers",
        "d_k",
        "Feed Forward Dimension",
        "Dropout",
        "Average Eval Loss"
    ],
    color_continuous_scale=custom_colors,
    range_color=[df_plot["Average Eval Loss"].min(), df_plot["Average Eval Loss"].max()]
)

fig.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black")
)

fig.show()


In [ ]:
import os
import re
import pandas as pd
import plotly.express as px

# Folder with loss CSVs
losses_folder = 'losses3'

# New filename pattern (no fold inside filename anymore)
filename_pattern = r'losses_n(\d+)_l(\d+)_dk(\d+)_dff(\d+)_do([\d\.]+)_fiveFold'

all_losses_data = []
for filename in os.listdir(losses_folder):
    if not filename.endswith(".csv"):
        continue
    match = re.search(filename_pattern, filename)
    if not match:
        continue

    n_heads = int(match.group(1))
    n_layers = int(match.group(2))
    d_k = int(match.group(3))
    dff = int(match.group(4))
    dropout = float(match.group(5))

    df_file = pd.read_csv(os.path.join(losses_folder, filename))
    df_file['n_heads'] = n_heads
    df_file['n_layers'] = n_layers
    df_file['d_k'] = d_k
    df_file['dff'] = dff
    df_file['dropout'] = dropout
    all_losses_data.append(df_file)

# Combine all loss files
df_combined = pd.concat(all_losses_data, ignore_index=True)

# Best val_loss per fold (inside each file)
df_best_per_fold = df_combined.loc[
    df_combined.groupby(['n_heads', 'n_layers', 'd_k', 'dff', 'dropout', 'fold'])['val_loss'].idxmin()
].reset_index(drop=True)

#Average best loss across folds
df_plot = df_best_per_fold.groupby(
    ['n_heads', 'n_layers', 'd_k', 'dff', 'dropout']
)['val_loss'].mean().reset_index()

# Rename final column
df_plot = df_plot.rename(columns={'val_loss': 'Average Eval Loss'})

# Rename hyperparameter columns
df_plot = df_plot.rename(columns={
    "n_heads": "Number of Heads",
    "n_layers": "Number of Layers",
    "d_k": "d_k",
    "dropout": "Dropout"
})

# Flag best configuration
best_idx = df_plot["Average Eval Loss"].idxmin()
df_plot["BestFlag"] = 0
df_plot.loc[best_idx, "BestFlag"] = 1

# Parallel coordinates plot WITHOUT Feed Forward Dimension
fig = px.parallel_coordinates(
    df_plot,
    color="BestFlag",
    dimensions=[
        "Number of Heads",
        "Number of Layers",
        "d_k",
        "Dropout",
        "Average Eval Loss"
    ],
    color_continuous_scale=[[0, "lightgray"], [1, "#9A031E"]],
    range_color=[0, 1]
)

fig.update_layout(
    autosize=False,
    width=1400,
     height=600,
    margin=dict(l=80, r=60, t=60, b=60),
    plot_bgcolor="white",
    font=dict(family="Arial", size=23, color="black"),coloraxis_showscale=False
)
fig.show()
